In [1]:
from bs4 import BeautifulSoup

In [2]:
import requests
import pandas as pd

In [3]:
requestsKamenev = pd.read_excel(r"C:\Users\HP\Notebooks\DataSets\RequestKamenev.xlsx")

Парсинг поисковых запросов Google

In [4]:
#YANDEX_RC_API_KEY = 'trnsl.1.1.20200219T191128Z.27cb3e2ca2fa6805.1f7c94fcbc20c5a639c46dd69f9fe503ddf1e7f3'
#YANDEX_RC_API_URL = 'http://rca.yandex.com/'
#URL = 'https://yandex.ru/search/?text=' + 'Каменев поэт'
#payload = {'key':YANDEX_RC_API_KEY, 'url': URL, 'content':'full', 'full':1}
#response = requests.get(YANDEX_RC_API_URL, params = payload)

In [5]:
def yandex_search(text, Frame): 
    frame = pd.DataFrame(columns=['Search Engine', 'Search Request', 'URL page'])
    frame = Frame
    url ='https://yandex.ru/search/?text=%(q)s'
    payload = {'q': text,}
    page = requests.get(url % payload) 
    soup = BeautifulSoup(page.text, 'lxml') 
    a = soup.find_all("a")
    for elem in a: 
        link =  elem.get('href')
        if(link.startswith("//") != -1 and link.find("webmaster") != -1 and link.find("images") != -1 
           and link.find("passport") != -1):
            line = {'Search Engine': 'Yandex' ,'IDRequest': i, 'URL page': link}
            frame = frame.append(line, ignore_index=True)
    return frame

In [7]:
def google_search(text, Frame): 
    frame = pd.DataFrame(columns=['Search Engine', 'Search Request', 'URL page'])
    frame = Frame
    url ='https://www.google.com/search?q='
    page = requests.get(url + text) 
    soup = BeautifulSoup(page.text) 
    a = soup.find_all("a")
    for elem in a: 
        hr = elem.get('href')
        if( hr.find("google") == -1 and hr.find("http") != -1 and (hr in frame) == False):
            link=("https://www.google.com" + elem.get('href'))
            line = {'Search Engine': 'Google' ,'IDRequest': i, 'URL page': link}
            frame = frame.append(line, ignore_index=True)
    return frame

Парсинг поисковых запросов Спутник

In [8]:
def sputnik_search(text, Frame):
    frame = pd.DataFrame(columns=['Search Engine', 'Search Request', 'URL page'])
    frame = Frame
    url ='http://www.sputnik.ru/search?q='
    page = requests.get(url + text)
    soup = BeautifulSoup(page.text) 
    div = soup.find_all("div", class_="b-result-title") 
    for elem in div: 
        elem=elem.contents[0]
        link=(elem["href"])
        line = {'Search Engine': 'Sputnik' ,'IDRequest': i, 'URL page': link}
        frame = frame.append(line, ignore_index=True)
    return frame

Парсинг поисковых запросов Webalta

In [9]:
def rambler_search(text, Frame): 
    frame = pd.DataFrame(columns=['Search Engine', 'Search Request', 'URL page'])
    frame = Frame
    url ='https://nova.rambler.ru/search?query='
    page = requests.get(url + text) 
    soup = BeautifulSoup(page.text) 
    a = soup.find_all("a") 
    for elem in a:      
        link=(elem["href"])
        if(link.find("ramble") == -1 and link.find("search") == -1 and link.find("url") == -1 
           and (link in frame['URL page']) == False):
            line = {'Search Engine': 'Rambler' ,'IDRequest': i, 'URL page': link}
            frame = frame.append(line, ignore_index=True) 
    return frame

In [10]:
def yahoo_search(text, Frame): 
    frame = pd.DataFrame(columns=['Search Engine', 'Search Request', 'URL page'])
    frame = Frame
    url ='https://ru.search.yahoo.com/search;_ylt=AwrJQ5vupEVecAYAJgXLxgt.;_ylc=X1MDMjExNDcwMTAwMwRfcgMyBGZyA3NmcARncHJpZANSVDh5eU4uTVFGTzVNdGMxdjRsTnVBBG5fcnNsdAMwBG5fc3VnZwMxMARvcmlnaW4DcnUuc2VhcmNoLnlhaG9vLmNvbQRwb3MDMARwcXN0cgMEcHFzdHJsAzAEcXN0cmwDNwRxdWVyeQNrZW1lcm92BHRfc3RtcAMxNTgxNjIyNTIx?p='
    page = requests.get(url + text) 
    soup = BeautifulSoup(page.text) 
    a = soup.find_all("a") 
    for elem in a:      
        link=(elem["href"])    
        if(link.find("tumblr") == -1 and link.find("yahoo") == -1 and link.find("void") == -1 and link.find("#") == -1):
            line = {'Search Engine': 'Yahoo' ,'IDRequest': i, 'URL page': link}
            frame = frame.append(line, ignore_index=True) 
    return frame

In [11]:
Frame = pd.DataFrame(columns=['Search Engine', 'IDRequest', 'URL page'])
for i in range(requestsKamenev.shape[0]):
    print(i)
    text = requestsKamenev['Searches'][i]
    Frame = google_search(text, Frame)
    Frame = sputnik_search(text, Frame)
    Frame = rambler_search(text, Frame)
    Frame = yahoo_search(text, Frame)
    Frame = yandex_search(text, Frame)
Frame.head()

0
[<a class="rui__30TPY" data-cerber-topline="projects::mail" href="https://mail.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=mail" rel="noopener noreferrer" target="_blank">Почта</a>, <a class="rui__30TPY" data-cerber-topline="projects::news_media" href="https://news.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=news_media" rel="noopener noreferrer" target="_blank">Новости</a>, <a class="rui__30TPY" data-cerber-topline="projects::kino_media" href="https://kino.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=kino_media" rel="noopener noreferrer" target="_blank">Кино</a>, <a class="rui__30TPY" data-cerber-topline="projects::sport_media" href="https://sport.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=sport_media" rel="noopener noreferrer" target="_blank">Спорт</a>, <a class="rui__30TPY"

1
[<a class="rui__30TPY" data-cerber-topline="projects::mail" href="https://mail.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=mail" rel="noopener noreferrer" target="_blank">Почта</a>, <a class="rui__30TPY" data-cerber-topline="projects::news_media" href="https://news.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=news_media" rel="noopener noreferrer" target="_blank">Новости</a>, <a class="rui__30TPY" data-cerber-topline="projects::kino_media" href="https://kino.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=kino_media" rel="noopener noreferrer" target="_blank">Кино</a>, <a class="rui__30TPY" data-cerber-topline="projects::sport_media" href="https://sport.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=sport_media" rel="noopener noreferrer" target="_blank">Спорт</a>, <a class="rui__30TPY"

2
[<a class="rui__30TPY" data-cerber-topline="projects::mail" href="https://mail.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=mail" rel="noopener noreferrer" target="_blank">Почта</a>, <a class="rui__30TPY" data-cerber-topline="projects::news_media" href="https://news.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=news_media" rel="noopener noreferrer" target="_blank">Новости</a>, <a class="rui__30TPY" data-cerber-topline="projects::kino_media" href="https://kino.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=kino_media" rel="noopener noreferrer" target="_blank">Кино</a>, <a class="rui__30TPY" data-cerber-topline="projects::sport_media" href="https://sport.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=sport_media" rel="noopener noreferrer" target="_blank">Спорт</a>, <a class="rui__30TPY"

3
[<a class="rui__30TPY" data-cerber-topline="projects::mail" href="https://mail.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=mail" rel="noopener noreferrer" target="_blank">Почта</a>, <a class="rui__30TPY" data-cerber-topline="projects::news_media" href="https://news.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=news_media" rel="noopener noreferrer" target="_blank">Новости</a>, <a class="rui__30TPY" data-cerber-topline="projects::kino_media" href="https://kino.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=kino_media" rel="noopener noreferrer" target="_blank">Кино</a>, <a class="rui__30TPY" data-cerber-topline="projects::sport_media" href="https://sport.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=sport_media" rel="noopener noreferrer" target="_blank">Спорт</a>, <a class="rui__30TPY"

4
[<a class="rui__30TPY" data-cerber-topline="projects::mail" href="https://mail.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=mail" rel="noopener noreferrer" target="_blank">Почта</a>, <a class="rui__30TPY" data-cerber-topline="projects::news_media" href="https://news.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=news_media" rel="noopener noreferrer" target="_blank">Новости</a>, <a class="rui__30TPY" data-cerber-topline="projects::kino_media" href="https://kino.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=kino_media" rel="noopener noreferrer" target="_blank">Кино</a>, <a class="rui__30TPY" data-cerber-topline="projects::sport_media" href="https://sport.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=sport_media" rel="noopener noreferrer" target="_blank">Спорт</a>, <a class="rui__30TPY"

...

18
[<a class="rui__30TPY" data-cerber-topline="projects::mail" href="https://mail.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=mail" rel="noopener noreferrer" target="_blank">Почта</a>, <a class="rui__30TPY" data-cerber-topline="projects::news_media" href="https://news.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=news_media" rel="noopener noreferrer" target="_blank">Новости</a>, <a class="rui__30TPY" data-cerber-topline="projects::kino_media" href="https://kino.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=kino_media" rel="noopener noreferrer" target="_blank">Кино</a>, <a class="rui__30TPY" data-cerber-topline="projects::sport_media" href="https://sport.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=sport_media" rel="noopener noreferrer" target="_blank">Спорт</a>, <a class="rui__30TPY

19
[<a class="rui__30TPY" data-cerber-topline="projects::mail" href="https://mail.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=mail" rel="noopener noreferrer" target="_blank">Почта</a>, <a class="rui__30TPY" data-cerber-topline="projects::news_media" href="https://news.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=news_media" rel="noopener noreferrer" target="_blank">Новости</a>, <a class="rui__30TPY" data-cerber-topline="projects::kino_media" href="https://kino.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=kino_media" rel="noopener noreferrer" target="_blank">Кино</a>, <a class="rui__30TPY" data-cerber-topline="projects::sport_media" href="https://sport.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=sport_media" rel="noopener noreferrer" target="_blank">Спорт</a>, <a class="rui__30TPY

20
[<a class="rui__30TPY" data-cerber-topline="projects::mail" href="https://mail.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=mail" rel="noopener noreferrer" target="_blank">Почта</a>, <a class="rui__30TPY" data-cerber-topline="projects::news_media" href="https://news.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=news_media" rel="noopener noreferrer" target="_blank">Новости</a>, <a class="rui__30TPY" data-cerber-topline="projects::kino_media" href="https://kino.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=kino_media" rel="noopener noreferrer" target="_blank">Кино</a>, <a class="rui__30TPY" data-cerber-topline="projects::sport_media" href="https://sport.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=sport_media" rel="noopener noreferrer" target="_blank">Спорт</a>, <a class="rui__30TPY

21
[<a class="rui__30TPY" data-cerber-topline="projects::mail" href="https://mail.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=mail" rel="noopener noreferrer" target="_blank">Почта</a>, <a class="rui__30TPY" data-cerber-topline="projects::news_media" href="https://news.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=news_media" rel="noopener noreferrer" target="_blank">Новости</a>, <a class="rui__30TPY" data-cerber-topline="projects::kino_media" href="https://kino.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=kino_media" rel="noopener noreferrer" target="_blank">Кино</a>, <a class="rui__30TPY" data-cerber-topline="projects::sport_media" href="https://sport.rambler.ru/?utm_source=search&amp;utm_campaign=self_promo&amp;utm_medium=topline&amp;utm_content=sport_media" rel="noopener noreferrer" target="_blank">Спорт</a>, <a class="rui__30TPY

,Search Engine,IDRequest,URL page
0,Google,0,https://www.google.com/imgres?imgurl=https://u...
1,Google,0,https://www.google.com/imgres?imgurl=http://az...
2,Google,0,https://www.google.com/imgres?imgurl=https://g...
3,Google,0,https://www.google.com/imgres?imgurl=https://i...
4,Google,0,https://www.google.com/url?q=https://ru.wikipe...


In [12]:
Frame.to_excel('urls.xlsx')